The code on this file was to create the rolling average dataset, that was used to solve the look-ahead bias problem we had.

In [ ]:
import numpy as np
import pandas as pd

# Load
df = pd.read_csv("Datasets/mlb_game_data_2025v2.csv")
df

,Date,Home Team,Away Team,Home homeruns,Away homeruns,Home hits,Away hits,Home obp,Away obp,Home slg,...,Away strikeouts,Home whip,Away whip,Home strikepercentage,Away strikepercentage,Home SP,Home SP ERA,Away SP,Away SP ERA,Home Team Won
0,"March 18, 2025",CHC,LAD,0.0,0.0,3.0,7.0,0.156,0.357,0.133,...,9.0,1.67,0.44,0.63,0.66,Shota Imanaga,0.00,Yoshinobu Yamamoto,1.80,False
1,"March 19, 2025",CHC,LAD,0.0,3.0,8.0,7.0,0.274,0.363,0.219,...,12.0,1.61,1.06,0.57,0.59,Justin Steele,11.25,Roki Sasaki,3.00,False
2,"March 27, 2025",NYY,MIL,2.0,1.0,7.0,7.0,0.324,0.289,0.483,...,10.0,1.22,1.38,0.65,0.60,Carlos Rodón,1.69,Freddy Peralta,3.60,True
3,"March 27, 2025",TOR,BAL,1.0,6.0,4.0,14.0,0.273,0.444,0.250,...,3.0,2.11,1.00,0.63,0.59,José Berríos,10.80,Zach Eflin,3.00,False
4,"March 27, 2025",TEX,BOS,0.0,2.0,7.0,6.0,0.257,0.250,0.242,...,7.0,1.00,1.00,0.67,0.68,Nathan Eovaldi,3.00,Garrett Crochet,3.60,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,"September 28, 2025",CLE,TEX,2.0,1.0,11.0,7.0,0.296,0.302,0.373,...,9.0,1.26,1.18,0.65,0.63,Logan Allen,8.31,Patrick Corbin,6.75,True
2426,"September 28, 2025",MIA,NYM,0.0,0.0,9.0,5.0,0.314,0.326,0.393,...,6.0,1.30,1.32,0.56,0.68,Edward Cabrera,0.00,Edwin Díaz,0.00,True
2427,"September 28, 2025",MIL,CIN,1.0,2.0,7.0,5.0,0.332,0.315,0.403,...,12.0,1.23,1.22,0.64,0.61,Aaron Ashby,0.00,Brady Singer,8.10,True
2428,"September 28, 2025",ATL,PIT,1.0,0.0,6.0,6.0,0.320,0.305,0.399,...,7.0,1.31,1.22,0.63,0.63,Chris Sale,1.59,Johan Oviedo,3.60,True


In [2]:
# Convert Date to datetime and sort properly
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values("Date")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")


Date range: 2025-03-18 00:00:00 to 2025-09-28 00:00:00


In [3]:
# Get all Home and Away stat columns (INCLUDING SP ERA now)
home_stats = [col for col in df.columns if col.startswith('Home ') and col not in ['Home Team', 'Home SP', 'Home Team Won']]
away_stats = [col for col in df.columns if col.startswith('Away ') and col not in ['Away Team', 'Away SP']]

# Create generic column names (remove 'Home ' and 'Away ' prefixes)
stat_names = [col.replace('Home ', '') for col in home_stats]

# Create home games dataframe for rolling calculation
home_games = df[['Date', 'Home Team'] + home_stats].copy()
home_games.columns = ['Date', 'Team'] + stat_names
home_games = home_games.sort_values(['Team', 'Date']).reset_index(drop=True)

# Create away games dataframe for rolling calculation
away_games = df[['Date', 'Away Team'] + away_stats].copy()
away_games.columns = ['Date', 'Team'] + stat_names
away_games = away_games.sort_values(['Team', 'Date']).reset_index(drop=True)

# Compute rolling 40-game averages (now including SP ERA)
rolling_cols = [col for col in stat_names]  # Include all stats now

# Compute rolling for home teams (SHIFT by 1 to use previous games only, not current)
home_rolling = home_games.groupby('Team')[rolling_cols].apply(
    lambda x: x.shift(1).rolling(window=40, min_periods=1).mean()
)
home_rolling = home_rolling.reset_index(level=0, drop=True)

# Compute rolling for away teams (SHIFT by 1 to use previous games only, not current)
away_rolling = away_games.groupby('Team')[rolling_cols].apply(
    lambda x: x.shift(1).rolling(window=40, min_periods=1).mean()
)
away_rolling = away_rolling.reset_index(drop=True)

# Add rolling stats with the ORIGINAL column names (not prefixed)
home_games_rolling = pd.concat([home_games[['Date', 'Team']].reset_index(drop=True), home_rolling], axis=1)
away_games_rolling = pd.concat([away_games[['Date', 'Team']].reset_index(drop=True), away_rolling], axis=1)

# Rename back to Home/Away format
home_rolling_renamed = home_games_rolling.drop(columns=['Date', 'Team']).add_prefix('Home ')
away_rolling_renamed = away_games_rolling.drop(columns=['Date', 'Team']).add_prefix('Away ')

# Add a row number within each team's games for accurate merging
home_games['home_game_num'] = home_games.groupby('Team').cumcount()
away_games['away_game_num'] = away_games.groupby('Team').cumcount()
home_rolling_renamed['home_game_num'] = home_games['home_game_num'].values
away_rolling_renamed['away_game_num'] = away_games['away_game_num'].values
home_rolling_renamed['Team'] = home_games['Team'].values
away_rolling_renamed['Team'] = away_games['Team'].values

# Add game numbers to original df
df_indexed = df.sort_values(['Date']).reset_index(drop=True)
df_indexed['home_game_num'] = df_indexed.groupby('Home Team').cumcount()
df_indexed['away_game_num'] = df_indexed.groupby('Away Team').cumcount()

# Start with identifier columns only (no SP names)
df_with_rolling = df_indexed[['Date', 'Home Team', 'Away Team', 'home_game_num', 'away_game_num']].copy()

# Merge home team rolling stats
df_with_rolling = df_with_rolling.merge(
    home_rolling_renamed,
    left_on=['Home Team', 'home_game_num'],
    right_on=['Team', 'home_game_num'],
    how='left'
).drop(columns=['Team'])

# Merge away team rolling stats
df_with_rolling = df_with_rolling.merge(
    away_rolling_renamed,
    left_on=['Away Team', 'away_game_num'],
    right_on=['Team', 'away_game_num'],
    how='left'
).drop(columns=['Team', 'home_game_num', 'away_game_num'])

# Add the target column at the end
df_with_rolling['Home Team Won'] = df_indexed['Home Team Won'].values

# Reorder columns to match original dataset order (but without SP names)
original_order = [col for col in df.columns if col not in ['Home SP', 'Away SP']]
df_with_rolling = df_with_rolling[original_order]

print(f"Original df shape: {df.shape}")
print(f"Df with rolling averages shape: {df_with_rolling.shape}")
print(f"\nColumns removed: Home SP, Away SP")
print(f"All stat columns (including SP ERA) now contain 40-game rolling averages")
print(f"\nColumn order preserved from original dataset")
print(f"\nSample data:")
df_with_rolling


Original df shape: (2430, 24)
Df with rolling averages shape: (2430, 22)

Columns removed: Home SP, Away SP
All stat columns (including SP ERA) now contain 40-game rolling averages

Column order preserved from original dataset

Sample data:


,Date,Home Team,Away Team,Home homeruns,Away homeruns,Home hits,Away hits,Home obp,Away obp,Home slg,...,Away leftonbase,Home strikeouts,Away strikeouts,Home whip,Away whip,Home strikepercentage,Away strikepercentage,Home SP ERA,Away SP ERA,Home Team Won
0,2025-03-18,CHC,LAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2025-03-19,CHC,LAD,0.000,0.000,3.000,7.000,0.156000,0.357000,0.133000,...,23.000,9.000,9.000,1.67000,0.44000,0.63000,0.66000,0.00000,1.80000,False
2,2025-03-27,NYY,MIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,2025-03-27,TOR,BAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2025-03-27,TEX,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,2025-09-28,MIA,NYM,1.100,1.400,7.900,8.025,0.314750,0.324500,0.393275,...,13.875,8.000,8.175,1.31975,1.31100,0.65575,0.64200,4.70075,5.09725,True
2426,2025-09-28,MIL,CIN,1.225,0.825,8.850,7.825,0.328425,0.317675,0.396125,...,13.350,10.300,8.500,1.24300,1.23750,0.64575,0.65150,3.71225,5.15700,True
2427,2025-09-28,LAA,HOU,1.175,1.250,7.275,9.250,0.302900,0.320075,0.405200,...,15.175,8.325,7.725,1.43375,1.20150,0.64475,0.62975,6.08000,4.69350,False
2428,2025-09-28,PHI,MIN,1.975,1.400,9.525,7.525,0.328250,0.311275,0.418450,...,13.875,10.200,8.075,1.24525,1.28325,0.65800,0.65225,3.29450,5.14550,True


In [4]:
df_with_rolling.to_csv("mlb_game_data_2025v2_with_rolling40.csv", index=False)

In [5]:
# Filter for Toronto Blue Jays home games
tor_home_games = df_with_rolling[df_with_rolling['Home Team'] == 'TOR']

print(f"Total TOR home games: {len(tor_home_games)}")
print(f"\nTOR home record: {tor_home_games['Home Team Won'].sum()} wins, {(~tor_home_games['Home Team Won']).sum()} losses")
print(f"Win percentage: {tor_home_games['Home Team Won'].mean():.3f}")

tor_home_games


Total TOR home games: 81

TOR home record: 54 wins, 27 losses
Win percentage: 0.667


,Date,Home Team,Away Team,Home homeruns,Away homeruns,Home hits,Away hits,Home obp,Away obp,Home slg,...,Away leftonbase,Home strikeouts,Away strikeouts,Home whip,Away whip,Home strikepercentage,Away strikepercentage,Home SP ERA,Away SP ERA,Home Team Won
3,2025-03-27,TOR,BAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
18,2025-03-28,TOR,BAL,1.000000,6.000000,4.000000,14.000000,0.273000,0.444000,0.250000,...,17.000000,8.000000,3.000000,2.110000,1.000000,0.630000,0.59000,10.80000,3.000000,True
29,2025-03-29,TOR,BAL,0.500000,3.500000,9.000000,8.500000,0.321500,0.393000,0.318500,...,13.500000,8.500000,6.000000,1.775000,1.265000,0.640000,0.60500,6.90000,6.900000,False
42,2025-03-30,TOR,BAL,0.666667,3.333333,8.666667,9.666667,0.332333,0.378667,0.344333,...,11.666667,8.666667,8.666667,1.663333,1.343333,0.643333,0.61000,6.60000,7.413333,True
60,2025-03-31,TOR,WSH,0.750000,NaN,8.250000,NaN,0.332500,NaN,0.356000,...,NaN,9.250000,NaN,1.595000,NaN,0.647500,NaN,5.32500,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,2025-09-24,TOR,BOS,1.175000,1.175000,9.175000,9.025000,0.333175,0.323900,0.416475,...,15.700000,8.825000,8.225000,1.260250,1.298500,0.641750,0.64425,4.55725,3.887250,False
2380,2025-09-25,TOR,BOS,1.175000,1.150000,8.950000,9.025000,0.333375,0.323950,0.417050,...,15.775000,8.725000,8.275000,1.261250,1.297750,0.642250,0.64500,4.69225,3.812250,True
2390,2025-09-26,TOR,TB,1.150000,1.175000,8.975000,7.800000,0.333575,0.316075,0.417625,...,12.875000,8.725000,9.325000,1.261750,1.209750,0.643500,0.64975,4.32875,4.742500,True
2407,2025-09-27,TOR,TB,1.175000,1.200000,8.975000,7.750000,0.333800,0.315875,0.418250,...,12.900000,8.600000,9.250000,1.262250,1.210000,0.643750,0.65175,3.96875,4.817500,True


In [7]:
# Count how many prior games each team has for each matchup
df_indexed_copy = df.sort_values(['Date']).reset_index(drop=True)
df_indexed_copy['home_games_before'] = df_indexed_copy.groupby('Home Team').cumcount()
df_indexed_copy['away_games_before'] = df_indexed_copy.groupby('Away Team').cumcount()

# Since we use .shift(1), home_games_before=0 means no prior home games
# home_games_before=1 means only 1 prior home game (not enough for good average)


# Check different minimum thresholds
for min_games in [0, 1, 5, 10, 20, 30, 40]:
    valid_games = df_indexed_copy[(df_indexed_copy['home_games_before'] >= min_games) & 
                                   (df_indexed_copy['away_games_before'] >= min_games)]
    print(f"\nMin {min_games} prior games: {len(valid_games)} games ({len(valid_games)/len(df_indexed_copy)*100:.1f}% of data)")



Min 0 prior games: 2430 games (100.0% of data)

Min 1 prior games: 2394 games (98.5% of data)

Min 5 prior games: 2257 games (92.9% of data)

Min 10 prior games: 2084 games (85.8% of data)

Min 20 prior games: 1799 games (74.0% of data)

Min 30 prior games: 1493 games (61.4% of data)

Min 40 prior games: 1198 games (49.3% of data)


In [8]:
# Create filtered dataset with min 10 prior games
MIN_PRIOR_GAMES = 10

df_indexed_filtered = df_indexed_copy[(df_indexed_copy['home_games_before'] >= MIN_PRIOR_GAMES) & 
                                       (df_indexed_copy['away_games_before'] >= MIN_PRIOR_GAMES)].copy()

# Now get the corresponding rows from df_with_rolling
# We'll use the index to match
df_with_rolling_filtered = df_with_rolling.loc[df_indexed_filtered.index].copy()

print(f"\n\nFILTERED TRAINING DATA:")
print(f"Original dataset: {len(df_with_rolling)} games")
print(f"Filtered dataset (min {MIN_PRIOR_GAMES} prior games): {len(df_with_rolling_filtered)} games")
print(f"Removed: {len(df_with_rolling) - len(df_with_rolling_filtered)} games ({(len(df_with_rolling) - len(df_with_rolling_filtered))/len(df_with_rolling)*100:.1f}%)")

# Verify no NaN values
nan_count = df_with_rolling_filtered.isna().sum().sum()
print(f"\nNaN values in filtered dataset: {nan_count}")

df_with_rolling_filtered



FILTERED TRAINING DATA:
Original dataset: 2430 games
Filtered dataset (min 10 prior games): 2084 games
Removed: 346 games (14.2%)

NaN values in filtered dataset: 0


,Date,Home Team,Away Team,Home homeruns,Away homeruns,Home hits,Away hits,Home obp,Away obp,Home slg,...,Away leftonbase,Home strikeouts,Away strikeouts,Home whip,Away whip,Home strikepercentage,Away strikepercentage,Home SP ERA,Away SP ERA,Home Team Won
235,2025-04-14,TB,BOS,1.000000,0.800000,8.833333,7.800000,0.329083,0.296400,0.393500,...,14.600000,9.500000,7.900000,0.999167,1.209000,0.661667,0.655000,2.901667,4.424000,True
241,2025-04-14,SD,CHC,1.000000,2.100000,8.900000,10.400000,0.347200,0.323400,0.438200,...,16.400000,10.100000,7.000000,1.080000,1.526000,0.639000,0.646000,2.137000,3.990000,True
250,2025-04-15,TB,BOS,1.230769,0.818182,9.384615,7.636364,0.329846,0.298636,0.396692,...,13.909091,9.846154,8.000000,1.006923,1.222727,0.667692,0.652727,2.793846,4.941818,False
255,2025-04-15,SD,CHC,1.181818,2.000000,9.181818,10.454545,0.347818,0.325545,0.438000,...,16.636364,10.000000,6.909091,1.079091,1.514545,0.639091,0.645455,2.231818,3.934545,False
256,2025-04-15,LAD,COL,1.800000,0.400000,7.400000,5.700000,0.316700,0.272300,0.465200,...,12.000000,10.700000,5.500000,1.258000,1.463000,0.630000,0.618000,1.800000,3.778000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,2025-09-28,MIA,NYM,1.100000,1.400000,7.900000,8.025000,0.314750,0.324500,0.393275,...,13.875000,8.000000,8.175000,1.319750,1.311000,0.655750,0.642000,4.700750,5.097250,True
2426,2025-09-28,MIL,CIN,1.225000,0.825000,8.850000,7.825000,0.328425,0.317675,0.396125,...,13.350000,10.300000,8.500000,1.243000,1.237500,0.645750,0.651500,3.712250,5.157000,True
2427,2025-09-28,LAA,HOU,1.175000,1.250000,7.275000,9.250000,0.302900,0.320075,0.405200,...,15.175000,8.325000,7.725000,1.433750,1.201500,0.644750,0.629750,6.080000,4.693500,False
2428,2025-09-28,PHI,MIN,1.975000,1.400000,9.525000,7.525000,0.328250,0.311275,0.418450,...,13.875000,10.200000,8.075000,1.245250,1.283250,0.658000,0.652250,3.294500,5.145500,True


In [9]:
df_with_rolling_filtered.to_csv("mlb_game_data_2025v2_with_rolling40_filtered.csv", index=False)